In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, when

# spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder \
    .master("local[1]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()


In [2]:
import pyspark.sql.functions as f

In [3]:
df = spark.read.json([
    "/home/jovyan/data/test.jsonl",
    "/home/jovyan/data/test2.jsonl"
])
df.select('steam_appid').distinct().count()

223874

In [4]:
df = df.distinct()
df.select('steam_appid').distinct().count()

223874

In [5]:
df.show(5)

+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+------------+--------------------+--------------------+--------------------+----------+-----------------------+--------------------+--------------------+--------------------+-------+--------------------+------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+----+--------------------+
|      about_the_game|        achievements|alternate_appid|          background|      background_raw|       capsule_image|     capsule_imagev5|          categories|co

In [6]:
df.printSchema()

root
 |-- about_the_game: string (nullable = true)
 |-- achievements: struct (nullable = true)
 |    |-- highlighted: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- path: string (nullable = true)
 |    |-- total: long (nullable = true)
 |-- alternate_appid: string (nullable = true)
 |-- background: string (nullable = true)
 |-- background_raw: string (nullable = true)
 |-- capsule_image: string (nullable = true)
 |-- capsule_imagev5: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |-- content_descriptors: struct (nullable = true)
 |    |-- ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- notes: string (nullable = true)
 |-- controller_support: string (nullable = true)
 |-- demos: array (

In [7]:
df.describe().show()


+-------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+------------------------+--------------------------+--------------------+---------------------+--------------------+--------------------+--------+--------------------+--------------------+------------------+------------------------+--------------------+------------------+--------------------+-----------+--------------------+
|summary|      about_the_game|   alternate_appid|          background|      background_raw|       capsule_image|     capsule_imagev5|controller_support|detailed_description|              drm_notice|   ext_user_account_notice|        header_image|         legal_notice|  linux_requirements|    mac_requirements|    name|     pc_requirements|             ratings|      required_age|                 reviews|   short_description|       steam_appid| supported_languages|       type|             website|


# Cleaning

In [8]:
df.groupBy("steam_appid").agg(f.count("*")).filter(col('count(1)')>1).show()

+-----------+--------+
|steam_appid|count(1)|
+-----------+--------+
|     412020|       2|
|        400|       2|
|      10180|       2|
|      33230|       2|
|      32460|       2|
|     244450|       2|
|      38480|       2|
|     311210|       3|
|    2195250|       3|
|      22330|       2|
|    2669320|       2|
|      34330|       2|
|     238210|       2|
|     359550|       2|
|        620|       2|
|      42680|       2|
|    1426210|       2|
+-----------+--------+



In [9]:
def find_differences_by_id(df, id_col):
    dup_ids = df.groupBy(id_col).count().filter(col("count") > 1).select(id_col)
    dup_df = df.join(dup_ids, on=id_col, how="inner")

    cols_to_check = [c for c in df.columns if c != id_col]

    diff_exprs = [
        f.collect_set(col(c)).alias(c) for c in cols_to_check
    ]

    grouped = dup_df.groupBy(id_col).agg(*diff_exprs)

    mismatch_exprs = [
        (f.size(col(c)) > 1).alias(c) for c in cols_to_check
    ]

    mismatches = grouped.select(id_col, *mismatch_exprs)

    mismatch_long = mismatches.selectExpr(
        f"`{id_col}`", 
        "stack(" + str(len(cols_to_check)) + ", " +
        ", ".join([f"'{c}', {c}" for c in cols_to_check]) +
        ") as (column, has_difference)"
    ).filter("has_difference = true")

    return mismatch_long


In [10]:
result = find_differences_by_id(df, "steam_appid")
result.show(100)

+-----------+--------------------+--------------+
|steam_appid|              column|has_difference|
+-----------+--------------------+--------------+
|      32460|      about_the_game|          true|
|      32460|          background|          true|
|      32460|      background_raw|          true|
|      32460|       capsule_image|          true|
|      32460|     capsule_imagev5|          true|
|      32460|          categories|          true|
|      32460|detailed_description|          true|
|      32460|        header_image|          true|
|      32460|                name|          true|
|      32460|      package_groups|          true|
|      32460|     pc_requirements|          true|
|      32460|             ratings|          true|
|      32460|         screenshots|          true|
|      32460|   short_description|          true|
|      32460|        support_info|          true|
|      32460| supported_languages|          true|
|      32460|                type|          true|


In [11]:
df.createOrReplaceTempView("df")


In [12]:
spark.sql("""
SELECT recommendations, ROW_NUMBER() OVER (PARTITION BY steam_appid ORDER BY recommendations DESC) AS rank FROM df WHERE steam_appid = '34330'
""").show()

+---------------+----+
|recommendations|rank|
+---------------+----+
|        {35065}|   1|
|        {35062}|   2|
+---------------+----+



In [13]:
df = spark.sql("""
SELECT *
FROM
(SELECT *, ROW_NUMBER() OVER (PARTITION BY steam_appid ORDER BY recommendations DESC) AS rank FROM df)
WHERE rank = 1
""").drop("rank")

In [14]:
result = find_differences_by_id(df, "steam_appid")
result.show(5)

+-----------+------+--------------+
|steam_appid|column|has_difference|
+-----------+------+--------------+
+-----------+------+--------------+



In [15]:
df.count()

223874

In [16]:
df.groupBy("steam_appid").agg(f.count("*")).filter(col('count(1)')>1).show()

+-----------+--------+
|steam_appid|count(1)|
+-----------+--------+
+-----------+--------+



In [17]:
def get_null_counts(df):
    null_counts = df.select([
        _sum(f.when(col(c).isNull(), 1).otherwise(0)).alias(c)
        for c in df.columns
    ])
    
    result = null_counts.selectExpr(
        "stack({0}, {1}) as (column_name, null_count)".format(
            len(df.columns),
            ", ".join([f"'{c}', `{c}`" for c in df.columns])
        )
    )
    
    return result.orderBy(f.desc("null_count"))


In [18]:
null_summary = get_null_counts(df)
null_summary = null_summary.withColumn("percent", col("null_count")/df.count())
null_summary.show(100)

+--------------------+----------+--------------------+
|         column_name|null_count|             percent|
+--------------------+----------+--------------------+
|     alternate_appid|    223860|  0.9999374648239635|
|          drm_notice|    222669|  0.9946175080625709|
|ext_user_account_...|    221832|  0.9908787978952447|
|          metacritic|    218735|  0.9770451235963087|
|             reviews|    210986|  0.9424319036600945|
|                 dlc|    207705|  0.9277763384761071|
|               demos|    204100|  0.9116735306467031|
|     recommendations|    202386|  0.9040174383805176|
|  controller_support|    168197|  0.7513020717010461|
|        achievements|    166183|  0.7423059399483638|
|        legal_notice|    153354|  0.6850013847074694|
|            fullgame|    141271|  0.6310290609896638|
|             website|    121069|  0.5407908019689647|
|              movies|     86407|  0.3859626396991165|
|      price_overview|     86339|  0.3856588974155105|
|         

In [19]:
null_columns = null_summary.filter('percent >=0.8').select('column_name').rdd.flatMap(lambda row:row).collect()

In [20]:
df = df.drop(*null_columns)

In [21]:
df.printSchema()

root
 |-- about_the_game: string (nullable = true)
 |-- achievements: struct (nullable = true)
 |    |-- highlighted: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- path: string (nullable = true)
 |    |-- total: long (nullable = true)
 |-- background: string (nullable = true)
 |-- background_raw: string (nullable = true)
 |-- capsule_image: string (nullable = true)
 |-- capsule_imagev5: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |-- content_descriptors: struct (nullable = true)
 |    |-- ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- notes: string (nullable = true)
 |-- controller_support: string (nullable = true)
 |-- detailed_description: string (nullable = true)
 |-- developer

In [22]:
df.select('achievements.highlighted.name').show()

+--------------------+
|                name|
+--------------------+
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|                NULL|
|[Defiant, Submiss...|
|[Someone Set Up U...|
|                NULL|
|[Double Cross, Ma...|
|                NULL|
|                NULL|
|                NULL|
|[Watch Your Head!...|
+--------------------+
only showing top 20 rows



In [23]:
df_cleaned = df.withColumn('num_achievements', f.array_size('achievements.highlighted.name'))

In [24]:
df.createOrReplaceTempView("df")

In [25]:
spark.sql("""
select category, count(*) from
    (select explode(categories.description) as category from df)
    group by category
    order by count(*) desc
""").show()

+--------------------+--------+
|            category|count(1)|
+--------------------+--------+
|       Single-player|  188762|
|      Family Sharing|  148055|
|  Steam Achievements|   90144|
|Full controller s...|   55681|
|         Steam Cloud|   53253|
|Downloadable Content|   51532|
|        Multi-player|   49361|
|Partial Controlle...|   31107|
|               Co-op|   30129|
|                 PvP|   25744|
| Steam Trading Cards|   24268|
|          Online PvP|   21999|
|           Game demo|   21947|
|        Online Co-op|   19714|
|  Steam Leaderboards|   19026|
| Shared/Split Screen|   16949|
|Remote Play Together|   12159|
|Cross-Platform Mu...|   11334|
|Shared/Split Scre...|   10056|
|Shared/Split Scre...|    9693|
+--------------------+--------+
only showing top 20 rows



In [26]:
spark.sql("""
select categories, genres from df
limit 1
""").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------+-------------+
|categories                                                                                                                          |genres       |
+------------------------------------------------------------------------------------------------------------------------------------+-------------+
|[{Multi-player, 1}, {PvP, 49}, {Online PvP, 36}, {Shared/Split Screen PvP, 37}, {Valve Anti-Cheat enabled, 8}, {Family Sharing, 62}]|[{Action, 1}]|
+------------------------------------------------------------------------------------------------------------------------------------+-------------+



In [27]:
spark.sql("""
select genre, count(*) from
    (select explode(genres.description) as genre from df)
    group by 1
    order by count(*) desc
""").show()

+--------------------+--------+
|               genre|count(1)|
+--------------------+--------+
|               Indie|  126028|
|              Action|   79642|
|              Casual|   72861|
|           Adventure|   71296|
|          Simulation|   44852|
|                 RPG|   43039|
|            Strategy|   41416|
|        Free To Play|   20107|
|        Early Access|   13952|
|              Sports|    8188|
|              Racing|    7171|
|Massively Multipl...|    5716|
|Design & Illustra...|    3533|
|      Web Publishing|    2846|
|           Utilities|    1972|
|           Education|    1660|
|    Game Development|    1583|
|Animation & Modeling|    1107|
|             Violent|     852|
|    Video Production|     730|
+--------------------+--------+
only showing top 20 rows



In [28]:
df.select("content_descriptors.notes").distinct().show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|notes                                                                                                                                                                                                                                                    |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Violence\r\nLanguage\r\nGambling                                                                                                                                                                                                                   

In [29]:
df.select('controller_support').distinct().show()

+------------------+
|controller_support|
+------------------+
|              full|
|              NULL|
+------------------+



In [30]:
df_cleaned = df_cleaned.withColumn('has_controller_support', when(col('controller_support') == 'full', True).otherwise(False)).drop('controller_support')

In [31]:
spark.sql("""
select fullgame.name, name from df where fullgame.name != name
""").show(truncate=False)

+-------------------------------+-----------------------------------+
|name                           |name                               |
+-------------------------------+-----------------------------------+
|Half-Life 2                    |Half-Life 2: Demo                  |
|Portal                         |Portal: First Slice                |
|Rag Doll Kung Fu               |Rag Doll Kung Fu Demo              |
|Darwinia                       |Darwinia Demo                      |
|DEFCON                         |DEFCON Demo                        |
|Multiwinia                     |Multiwinia Demo                    |
|Arx Fatalis                    |Arx Fatalis Demo                   |
|Shadowgrounds                  |Shadowgrounds Demo                 |
|ThreadSpace: Hyperbol          |ThreadSpace: Hyperbol Demo         |
|King's Bounty: Armored Princess|King's Bounty Armored Princess Demo|
|Uninitialized                  |Painkiller Demo                    |
|Painkiller Overdose

In [32]:
df_cleaned = df_cleaned.withColumn('num_packages', f.array_size('packages'))

In [36]:
 df_cleaned = df_cleaned.withColumns({'is_mac_supported':df_cleaned.platforms.mac, 'is_linux_supported':df_cleaned.platforms.linux,'is_windows_supported':df_cleaned.platforms.windows})

In [37]:
df_cleaned.select('price_overview.currency').distinct().show()

+--------+
|currency|
+--------+
|     PHP|
|    NULL|
+--------+



In [38]:
df_cleaned.select('price_overview.discount_percent').distinct().show()

+----------------+
|discount_percent|
+----------------+
|              26|
|              29|
|              65|
|              19|
|              54|
|               0|
|              22|
|              77|
|              34|
|              50|
|              94|
|              57|
|              43|
|              32|
|              84|
|              31|
|              39|
|              25|
|              95|
|              71|
+----------------+
only showing top 20 rows



In [39]:
df_cleaned.select('price_overview.recurring_sub').distinct().show()

+-------------+
|recurring_sub|
+-------------+
|       705729|
|       774628|
|       214296|
|       979485|
|       652938|
|       212320|
|       350099|
|       543658|
|       264676|
|       397465|
|       241413|
|       157579|
|       241397|
|       425251|
|       700793|
|      1062615|
|       496383|
|       282166|
|       282709|
|       244138|
+-------------+
only showing top 20 rows



In [40]:
df_cleaned.select('price_overview.recurring_sub_desc').distinct().show(truncate=False)

+-----------------------------------------------------------------------------------------------------+
|recurring_sub_desc                                                                                   |
+-----------------------------------------------------------------------------------------------------+
|P142.00 at checkout, auto-renewed every 1 month(s) at P142.00.                                       |
|P35.95 at checkout, auto-renewed every 1 month(s) at P109.95.                                        |
|P319.95 at checkout, auto-renewed every 1 month(s) at P239.96.                                       |
|P319.95 at checkout, auto-renewed every 1 month(s) at P319.95.                                       |
|P34.95 at checkout, auto-renewed every 1 month(s) at P34.95.                                         |
|P1,115.00 at checkout, auto-renewed every 1 month(s) at P1,115.00 as determined by your game account.|
|P149.00 at checkout, auto-renewed every 1 month(s) at P149.00. 

In [41]:
spark.sql("""
select publisher, count(*) from
    (select explode(publishers) as publisher from df)
    group by publisher
    order by count(*) desc
""").show()

+--------------------+--------+
|           publisher|count(1)|
+--------------------+--------+
|         TigerQiuQiu|    2305|
|              KOMODO|    2287|
|KOEI TECMO GAMES ...|    1451|
|           N3V Games|     864|
|    CAPCOM Co., Ltd.|     812|
|                SEGA|     663|
|Dovetail Games - ...|     646|
| Paradox Interactive|     592|
| Gotcha Gotcha Games|     589|
|      Big Fish Games|     520|
|BANDAI NAMCO Ente...|     513|
|        PlayWay S.A.|     511|
|         Square Enix|     502|
|         XSEED Games|     458|
| Marvelous USA, Inc.|     441|
|          THQ Nordic|     441|
|             Ubisoft|     399|
|     Electronic Arts|     387|
|     Choice of Games|     386|
|   NIS America, Inc.|     368|
+--------------------+--------+
only showing top 20 rows



In [42]:
df_cleaned.select('ratings').distinct().show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ratings                                                                                                                                                                                                                                                                                                                                                 

In [61]:
schema = MapType(StringType(), MapType(StringType(), StringType()))
df_cleaned = df.withColumn("parsed_ratings", f.from_json(col("ratings"), schema))

In [62]:
df_cleaned.select(explode(map_keys(col("parsed_ratings")))) \
              .distinct() \
              .show()

+-------------+
|          col|
+-------------+
|          crl|
|         kgrb|
|        dejus|
|         cero|
|       nzoflc|
|         pegi|
|         oflc|
|          usk|
|          fpb|
|         csrr|
|        cadpa|
|steam_germany|
|         bbfc|
|        video|
|        agcom|
|          mda|
|         esrb|
+-------------+



In [64]:
ratings = df_cleaned.select(explode(map_keys(col("parsed_ratings")))) \
              .distinct() \
              .collect()

In [73]:
rating_systems = [row.col for row in ratings]
rating_systems

['crl',
 'kgrb',
 'dejus',
 'cero',
 'nzoflc',
 'pegi',
 'oflc',
 'usk',
 'fpb',
 'csrr',
 'cadpa',
 'steam_germany',
 'bbfc',
 'video',
 'agcom',
 'mda',
 'esrb']

In [74]:
df_cleaned = df_cleaned.withColumns({rating: col('parsed_ratings')[rating] for rating in rating_systems})

In [49]:
df_cleaned.select(f.to_date('release_date.date', 'd MMM, yyyy')).distinct().show(truncate=False)

+---------------------------------------+
|to_date(release_date.date, d MMM, yyyy)|
+---------------------------------------+
|2007-04-20                             |
|2012-04-17                             |
|2013-03-26                             |
|2013-05-21                             |
|2013-09-09                             |
|2013-01-22                             |
|2014-09-26                             |
|2021-06-22                             |
|2014-11-12                             |
|2015-05-19                             |
|2015-03-09                             |
|2018-08-10                             |
|2016-03-01                             |
|2017-08-11                             |
|2018-05-28                             |
|2017-09-11                             |
|2024-09-18                             |
|2022-03-28                             |
|2021-08-27                             |
|2019-06-04                             |
+---------------------------------

In [56]:
df_cleaned = df.withColumn('release_date', f.to_date(df.release_date.date, 'd MMM, yyyy'))

In [58]:
df_cleaned.select(f.max(f.to_date('release_date', 'd MMM, yyyy'))).show()

+---------------------------------------+
|max(to_date(release_date, d MMM, yyyy))|
+---------------------------------------+
|                             7777-03-31|
+---------------------------------------+



In [59]:
df_cleaned.select(f.min(f.to_date('release_date', 'd MMM, yyyy'))).show()

+---------------------------------------+
|min(to_date(release_date, d MMM, yyyy))|
+---------------------------------------+
|                             1969-12-31|
+---------------------------------------+



In [65]:
df_cleaned.select('required_age').distinct().show()

+------------+
|required_age|
+------------+
|           7|
|          15|
|          11|
|           3|
|           8|
|          16|
|           0|
|           5|
|         18+|
|          18|
|          17|
|           6|
|        １８|
|           1|
|          7+|
|          20|
|          10|
|           4|
|          12|
|          13|
+------------+
only showing top 20 rows



In [66]:
df_cleaned.select('type').distinct().show()

+-----------+
|       type|
+-----------+
|advertising|
|        dlc|
|    episode|
|      music|
|     series|
|      video|
|   hardware|
|       demo|
|       game|
|        mod|
+-----------+



In [69]:
df_cleaned.select('supported_languages').distinct().show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|supported_languages                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------

# EDA

In [ ]:
df_cleaned.groupBy('type').agg(f.count("*").alias('counts')).orderBy('counts', ascending=False).show()

In [ ]:
df.explain(mode="formatted")